<a href="https://colab.research.google.com/github/okothchristopher/introtodeeplearning/blob/master/deep_learning_practice_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# deep learning with keras first practice 

In [7]:
#  import the libraries I need 
import pandas as pd
from keras.layers import Dense 
from keras.models import Sequential # we will be using sequential models for this first practice 
from sklearn.model_selection import train_test_split

In [4]:
df=pd.read_csv('https://assets.datacamp.com/production/repositories/654/datasets/8a57adcdb5bfb3e603dad7d3c61682dfe63082b8/hourly_wages.csv')
df.head()

,wage_per_hour,union,education_yrs,experience_yrs,age,female,marr,south,manufacturing,construction
0,5.10,0,8,21,35,1,1,0,1,0
1,4.95,0,9,42,57,1,1,0,1,0
2,6.67,0,12,1,19,0,0,0,1,0
3,4.00,0,12,4,22,0,0,0,0,0
4,7.50,0,12,17,35,0,1,0,0,0


In [5]:
# so we want to predict wage per hour based on the attributes of the worker 
df.describe()

,wage_per_hour,union,education_yrs,experience_yrs,age,female,marr,south,manufacturing,construction
count,534.000000,534.000000,534.000000,534.000000,534.000000,534.000000,534.000000,534.000000,534.000000,534.000000
mean,9.024064,0.179775,13.018727,17.822097,36.833333,0.458801,0.655431,0.292135,0.185393,0.044944
std,5.139097,0.384360,2.615373,12.379710,11.726573,0.498767,0.475673,0.455170,0.388981,0.207375
min,1.000000,0.000000,2.000000,0.000000,18.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,5.250000,0.000000,12.000000,8.000000,28.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,7.780000,0.000000,12.000000,15.000000,35.000000,0.000000,1.000000,0.000000,0.000000,0.000000
75%,11.250000,0.000000,15.000000,26.000000,44.000000,1.000000,1.000000,1.000000,0.000000,0.000000
max,44.500000,1.000000,18.000000,55.000000,64.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [8]:
# X=df.drop(['wage_per_hour'],axis=1)
# y=df['wage_per_hour']
# from sklearn.preprocessing import StandardScaler # it is always good practice to scale data before training a neural network 
# scaler=StandardScaler()
# X=scaler.fit_transform(X)
# X.shape
X_train,X_test,y_train,y_test=train_test_split(X,y)
print(X_train.shape,y_train.shape,X_test.shape,y_test.shape)


(400, 9) (400,) (134, 9) (134,)


In [63]:
# modelling 
n_cols=X.shape[1] # this will define the input nodes 
model=Sequential()
model.add(Dense(100,activation='relu',input_shape=(n_cols,)))#after specifying the input shape parameter in the first layer then we can do an input shape reference 
model.add(Dense(50,activation='relu'))
model.add(Dense(20,activation='relu'))
model.pop()
model.add(Dense(1))
model.compile(optimizer='sgd',loss='mean_squared_error',metrics=['mean_squared_error'])
model.fit(X_train,y_train,epochs=20)

Epoch 1/20
13/13 [==============================] - 0s 3ms/step - loss: 44.9580 - mean_squared_error: 44.9580
Epoch 2/20
13/13 [==============================] - 0s 2ms/step - loss: 18.3664 - mean_squared_error: 18.3664
Epoch 3/20
13/13 [==============================] - 0s 2ms/step - loss: 16.4561 - mean_squared_error: 16.4561
Epoch 4/20
13/13 [==============================] - 0s 2ms/step - loss: 16.1597 - mean_squared_error: 16.1597
Epoch 5/20
13/13 [==============================] - 0s 3ms/step - loss: 15.6161 - mean_squared_error: 15.6161
Epoch 6/20
13/13 [==============================] - 0s 2ms/step - loss: 15.5259 - mean_squared_error: 15.5259
Epoch 7/20
13/13 [==============================] - 0s 3ms/step - loss: 14.7053 - mean_squared_error: 14.7053
Epoch 8/20
13/13 [==============================] - 0s 3ms/step - loss: 15.3305 - mean_squared_error: 15.3305
Epoch 9/20
13/13 [==============================] - 0s 2ms/step - loss: 14.8076 - mean_squared_error: 14.8076
Epoch 10/2

In [64]:
model.summary()

Model: "sequential_45"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_139 (Dense)            (None, 100)               1000      
_________________________________________________________________
dense_140 (Dense)            (None, 50)                5050      
_________________________________________________________________
dense_142 (Dense)            (None, 1)                 51        
Total params: 6,101
Trainable params: 6,101
Non-trainable params: 0
_________________________________________________________________


In [65]:
y_pred_1=model.predict(X_test)

In [66]:
from sklearn import metrics
metrics.mean_squared_error(y_test,y_pred_1)

29.039182781862273

In [26]:
# let us compare this with random forest 
from sklearn.ensemble import RandomForestRegressor
model_2=RandomForestRegressor()
model_2.fit(X_train,y_train)
y_pred=model_2.predict(X_test)
metrics.mean_squared_error(y_test,y_pred)

30.897251813389616

In [49]:
# adam is an optimization method that relies on the adaptive estimation of the first and ssecond order moments ,it is a sgd method 
# with deep learning there is no need for cross validation why?


In [59]:
# then let us optimize this model and see if it can improve 
## stochastic gradient descent 
def get_new_model(input_shape=(n_cols,)):
  model=Sequential()
  model.add(Dense(120,activation="relu",input_shape=input_shape))
  model.add(Dense(60,activation="relu"))
  model.add(Dense(50,activation="relu"))
  model.add(Dense(1,activation='relu'))# since this is a regression problem the class of the output is just one 
  return model

lr_to_test=[0.001,0.0001,0.1,0.003,0.3,0.0004,0.000001,0.0005]
from keras.optimizers import  SGD

for lr in lr_to_test:
  model=get_new_model()
  optimizer=SGD(lr=lr)
  model.compile(optimizer=optimizer,loss='mean_squared_error')
  model.fit(X_train,y_train)

13/13 [==============================] - 0s 2ms/step - loss: 100.9312


In [67]:
## early stopping 
from keras.callbacks import EarlyStopping
early_stopper=EarlyStopping(patience=2)# stop the model once the metric does not improve past 2
model.fit(X,y,validation_split=.3,epochs=20,
          callbacks=[early_stopper])


Epoch 1/20
12/12 [==============================] - 0s 13ms/step - loss: 17.4984 - mean_squared_error: 17.4984 - val_loss: 22.0279 - val_mean_squared_error: 22.0279
Epoch 2/20
12/12 [==============================] - 0s 5ms/step - loss: 15.8928 - mean_squared_error: 15.8928 - val_loss: 21.7391 - val_mean_squared_error: 21.7391
Epoch 3/20
12/12 [==============================] - 0s 5ms/step - loss: 15.9576 - mean_squared_error: 15.9576 - val_loss: 24.1563 - val_mean_squared_error: 24.1563
Epoch 4/20
12/12 [==============================] - 0s 5ms/step - loss: 16.5562 - mean_squared_error: 16.5562 - val_loss: 22.9093 - val_mean_squared_error: 22.9093


In [ ]:
# notes-creating a model requires experimentation 
# increasing the number of epochs can increase the model perfomance and to this end one needs an early stoppage 
